In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Распознавание лиц при помощи SVM и kernel trick

# Описание набора данных

В этом задании вам предлагается построить классификатор, обученный на изображениях лиц. Сначала загрузим необходимый набор данных и посмотрим, что он из себя представляет. 

Таблица содержит "распрямленные" представления изображений лиц людей.

Изначально каждое изображение — матрица размера $62 \times 47$, значения которой, предположительно, нормированы к диапазону $[0, 1]$ из диапазона $[0, 255] \cap \mathbb Z$. Сами значения отвечают за интенсивность пикселя: чем значение больше, тем пиксель светлее.

Обратите внимание, из картинки (или, что то же самое, матрицы) размера $62 \times 47$ получился вектор длины $2914 = 62 \cdot 47$.

Колонка <code>label</code> отвечает за имя человека, изображенного на картинке.

In [ ]:
df = pd.read_csv('persons_pics_train.csv')
df

Убедимся в том, что перед нами действительно изображения лиц людей. Возьмем первые $15$ строк из таблицы исходных данных и построим соответствующие им изображения.

In [ ]:
import matplotlib.pyplot as plt

def get_img_by_row(row):
  return row.drop('label').astype(float).to_numpy().reshape(62,47), row['label']

In [ ]:
fig, ax = plt.subplots(3, 6)
for i, axi in enumerate(ax.flat):
    img, lbl = get_img_by_row(df.iloc[i])
    axi.imshow(img, cmap='gray')
    axi.set(xticks=[], yticks=[],
            xlabel=lbl.split()[-1])
plt.savefig('persons_pics_img_for_description.png', dpi = 300, bbox_inches='tight')    

Крупнее

In [ ]:
plt.imshow(df.iloc[0].drop('label').astype(float).to_numpy().reshape(62,47), cmap='gray')
plt.axis('off')

# Немного озорства

Определите число различных людей, фотографии которых содержатся в рассматриваемом наборе данных.

In [ ]:
df['label'].groupby(df['label']).count().count()

Постройте гистограмму распределения объектов по классам и вычислите долю объектов каждого класса. 

In [ ]:
hist = df['label'].groupby(df['label']).count().plot(kind='bar')
plt.show(hist)

In [ ]:
df[df['label'] == 'Junichiro Koizumi'].shape[0] / df.shape[0]

На основе имеющихся данных поставьте в соответствие каждому человеку один вектор, координаты которого являются средним соответствующих координат всех векторов, отождествленных с этим человеком.

In [ ]:
persons = df['label'].unique()
persons

In [ ]:
averaged_vector = pd.DataFrame(columns=persons)
for p in persons:
  averaged_vector[p] = df[df['label'] == p].drop('label', axis=1).mean(axis=0)
averaged_vector

Вычислите координату с индексом $0$ усредненного вектора, соответствующего человеку из вашего индивидуального задания.

In [ ]:
averaged_vector['Hugo Chavez'][0]

Используя усредненные векторы, постройте изображения всех людей из набора данных.

In [ ]:
import matplotlib.pyplot as plt

def get_img_by_name(name):
  return name.astype(float).to_numpy().reshape(62,47)

In [ ]:
pic = get_img_by_name(averaged_vector['Hugo Chavez'])
plt.imshow(pic, cmap='gray')

In [ ]:
fig, ax = plt.subplots(3, 4)
for i, axi in enumerate(ax.flat):
    lbl = persons[i]
    img = get_img_by_name(averaged_vector[persons[i]])
    axi.imshow(img, cmap='gray')
    axi.set(xticks=[], yticks=[],
            xlabel=lbl.split()[-1])
plt.savefig('persons_pics_img_for_description.png', dpi = 300, bbox_inches='tight')    

В качестве меры "схожести" двух объектов часто используют так называемое [косинусное сходство](https://en.wikipedia.org/wiki/Cosine_similarity). Не следует путать с косинусным расстоянием.

Постройте матрицу "схожести" объектов на основе их усредненных векторов. Для визуализации рекомендуем использовать тепловую карту, например, [из библиотеки seaborn](https://seaborn.pydata.org/generated/seaborn.heatmap.html).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(averaged_vector.T.to_numpy())

In [ ]:
cosine_similarity_matrix.shape

In [ ]:
import seaborn as sns

sns.heatmap(cosine_similarity_matrix)
plt.show()


In [ ]:
# The value of the cosine similarity between a pair of averaged vectors Jacques Chiracand Serena Williams.
cosine_similarity_matrix[11][10]

Определите косинусное сходство для пары людей из вашего индивидуального задания на основе соответствующих им "усредненных" векторов.

In [ ]:
# < ENTER YOUR CODE HERE > 

# Построение модели

При помощи <code>train_test_split()</code> разделите набор данных на тренировочный и тестовый с параметрами, указанными в вашем индивидуальном задании. Используйте стратификацию по колонке <code>label</code>.

In [ ]:
# import train_test_split
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1).to_numpy()
y = df['label'].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)


## Первое приближение: базовый SVM классификатор с линейным ядром.

Обучите модель <code>SVC()</code> на тренировочном наборе данных с параметрами, указанными в вашем индивидуальном задании.

In [ ]:
# create a SVC classifier
from sklearn.svm import SVC

clf = SVC(kernel='linear', random_state=13)
clf.fit(x_train, y_train)

Оцените модель на тестовых данных. Вычислите <code>f1_score(average='weighted')</code>

In [ ]:
# import f1_score
from sklearn.metrics import f1_score

f1_score(y_test, clf.predict(x_test), average='weighted')

В целом, достаточно неплохой результат, но имеет смысл попробовать подобрать параметры.

## Подбор гиперпараметров

Используя <code>GridSearchCV()</code>, осуществите подбор гиперпараметров, указанных в вашем задании. Отдельно обратите внимание на время обучения моделей

Определите гиперпараметры лучшей полученной модели.

In [ ]:
from sklearn.model_selection import GridSearchCV
tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000], 'class_weight': [None, 'balanced'], 'random_state':[13]}]
cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3)

In [ ]:
cv.fit(x_train, y_train)

In [ ]:
cv.best_params_

Выполните предсказание для тестовых данных при помощи лучшей полученной модели. Вычислите <code>f1_score(average='weighted')</code>.

In [ ]:
f1_score(y_test, cv.predict(x_test), average='weighted')

0.8153472449345439

## Уменьшение размерности

Обучение было долгим. Имеет смысл проверить, что будет если уменьшить размерность входных данных, например, при помощи <code>PCA</code>. Обучите модель <code>PCA(svd_solver='full')</code> на тренировочных данных и определите минимальное количество компонент, необходимое для того, чтобы уровень объясненной дисперсии был больше <code>0.95</code>.

In [81]:
# train the model PCA(svc_solver='full')
from sklearn.decomposition import PCA
pca = PCA(svd_solver='full')
pca.fit(x_train)

PCA(svd_solver='full')

In [93]:
explained_variances = pca.explained_variance_ratio_
cumulative_variances = np.cumsum(explained_variances)
num_components_needed = np.argmax(cumulative_variances > 0.95) + 1
print("The minimum number of components required for the level of explained variance to be greater than 0.95 is:", num_components_needed)


The minimum number of components required for the level of explained variance to be greater than 0.95 is: 173


Заново обучите модель <code>PCA(svd_solver='full')</code> на тренировочных данных с использованием полученного числа ГК. Примените полученное преобразование для тренировочных и тестовых данных.

In [95]:
pca = PCA(n_components=num_components_needed, svd_solver='full')
X_train_pca = pca.fit_transform(x_train)
X_test_pca = pca.transform(x_test)

Для полученных после <code>PCA</code> данных аналогично проделанному ранее, то есть используя <code>GridSearchCV()</code>, осуществите подбор гиперпараметров, указанных в вашем задании. Отдельно обратите внимание на время обучения моделей.

In [96]:
cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3)
cv.fit(X_train_pca, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.774 total time=   0.1s
[CV 2/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.811 total time=   0.1s
[CV 3/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.778 total time=   0.1s
[CV 4/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.764 total time=   0.1s
[CV 5/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.722 total time=   0.1s
[CV 1/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.340 total time=   0.1s
[CV 2/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.340 total time=   0.1s
[CV 3/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.340 total time=   0.1s
[CV 4/5] END C=1, class_weight=N

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'class_weight': [None, 'balanced'],
                          'gamma': [0.001, 0.0001],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                          'random_state': [13]}],
             verbose=3)

Определите гиперпараметры лучшей полученной модели.

In [97]:
cv.best_params_

{'C': 100,
 'class_weight': None,
 'gamma': 0.001,
 'kernel': 'sigmoid',
 'random_state': 13}

Выполните предсказание для тестовых данных при помощи лучшей полученной модели. Вычислите <code>f1_score(average='weighted')</code>

In [98]:
f1_score(y_test, cv.predict(X_test_pca), average='weighted')

0.8147394395247998

Видно, что качество модели изменилось незначительно, при этом время обучения существенно сократилось.